In [1]:
from torchvision import transforms
from PIL import Image
from typing import Any

import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from torchvision.transforms import ToPILImage

from anomalib import TaskType
from anomalib.data import MVTec
from anomalib.data.utils import read_image
from anomalib.deploy import ExportType, OpenVINOInferencer
from anomalib.engine import Engine
from anomalib.models import Padim
import os
from pathlib import Path

In [2]:
pwd

'd:\\Shravtek\\neutech_airfillter\\anamolib'

In [3]:
cd anomalib/

d:\Shravtek\neutech_airfillter\anamolib\anomalib


In [4]:
pwd

'd:\\Shravtek\\neutech_airfillter\\anamolib\\anomalib'

In [5]:
# Import the datamodule
from anomalib.data import Folder
from anomalib.data.utils import TestSplitMode

# Create the datamodule
datamodule = Folder(
    name="toy",
    root="datasets/hazelnut_toy",
    normal_dir="good",
    test_split_mode=TestSplitMode.SYNTHETIC,
    task="classification",
    image_size=(512,512)
  )

# Setup the datamodule
datamodule.setup()

In [6]:
i, train_data = next(enumerate(datamodule.train_dataloader()))
print(train_data.keys())
# dict_keys(['image_path', 'label', 'image'])

i, val_data = next(enumerate(datamodule.val_dataloader()))
print(val_data.keys())
# dict_keys(['image_path', 'label', 'image'])

i, test_data = next(enumerate(datamodule.test_dataloader()))
print(test_data.keys())
# dict_keys(['image_path', 'label', 'image'])

dict_keys(['image_path', 'label', 'image'])
dict_keys(['image_path', 'label', 'image'])
dict_keys(['image_path', 'label', 'image'])


In [7]:
# Import the model and engine
from anomalib.models import Patchcore
from anomalib.engine import Engine
from anomalib.callbacks import ModelCheckpoint
from anomalib.loggers import AnomalibCometLogger
from anomalib.loggers import AnomalibTensorBoardLogger
from anomalib.loggers import AnomalibWandbLogger


Tenlogger = AnomalibWandbLogger( project="my-awesome-project",config={
    "architecture": "Patchcore",
    "dataset": "hazelnut_toy",
    "epochs": 3,})
callbacks = [ModelCheckpoint()]
# Create the model and engine
model = Patchcore()
engine = Engine(task="classification",
    accelerator="auto",
    check_val_every_n_epoch=1,
    max_epochs=2,
    num_sanity_val_steps=0,
    val_check_interval=1.0,
    callbacks=callbacks,
    logger=Tenlogger)


In [8]:

# Train a Patchcore model on the given datamodule
engine.train(datamodule=datamodule, model=model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shan1023. Use `wandb login --relogin` to force relogin


F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
c:\Users\shan\.conda\envs\anamo\lib\site-packages\lightning\pytorch\core\optimizer.py:180: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name                  | Type                     | Params
-------------------------------------------------------------------
0 | model                 | PatchcoreModel           | 24.9 M
1 | _transform            | Compose                  | 0     
2 | normalization_metrics | MinMax                   | 0     
3 | image_threshold       | F1AdaptiveThreshold      | 0     
4 | pixel_threshold       | F1AdaptiveThreshold      | 0     
5 | image_metrics         | AnomalibMetricCollection | 0     
6 | pixel_metrics         | AnomalibMetricCollection | 0     
-------------------------------------------------------------------
24.9 M    Trainable params
0         Non-trainable para

Training: |          | 0/? [00:00<?, ?it/s]

c:\Users\shan\.conda\envs\anamo\lib\site-packages\lightning\pytorch\loops\optimization\automatic.py:129: `training_step` returned `None`. If this was on purpose, ignore this warning...


Validation: |          | 0/? [00:00<?, ?it/s]

Output()

`Trainer.fit` stopped: `max_epochs=1` reached.
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
c:\Users\shan\.conda\envs\anamo\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │            1.0            │
│       image_F1Score       │            1.0            │
└───────────────────────────┴───────────────────────────┘

In [ ]:
# tensorboard --logdir="tb_logs/my_model/version_0/events.out.tfevents.1712825887.DESKTOP-0JKHALG.8636.4"

In [20]:
# comet_logger.experiment.end()

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml OfflineExperiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : [OfflineExperiment will get URL after upload]
COMET INFO:   Others:
COMET INFO:     Created from       : pytorch-lightning
COMET INFO:     offline_experiment : True
COMET INFO:   Uploads:
COMET INFO:     conda-environment-definition : 1
COMET INFO:     conda-info                   : 1
COMET INFO:     conda-specification          : 1
COMET INFO:     environment 

In [9]:
engine.export(
    model=model,
    export_type=ExportType.OPENVINO,
)

c:\Users\shan\.conda\envs\anamo\lib\site-packages\anomalib\data\transforms\center_crop.py:57: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if image.numel() == 0:
c:\Users\shan\.conda\envs\anamo\lib\site-packages\anomalib\data\transforms\center_crop.py:61: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if crop_height > image_height or crop_width > image_width:
c:\Users\shan\.conda\envs\anamo\lib\site-packages\anomalib\data\transforms\center_crop.py:41: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore th

WindowsPath('D:/Shravtek/neutech_airfillter/anamolib/anomalib/results/Patchcore/bottle/latest/weights/openvino/model.xml')

In [32]:
image_path = "D:/Shravtek/neutech_airfillter/anamolib/anomalib/datasets/bottle/good/010.png"
image = read_image(path=image_path)
plt.imshow(image)

In [20]:
output_path = Path(engine.trainer.default_root_dir)
print(output_path)

D:\Shravtek\neutech_airfillter\anamolib\anomalib\results\Patchcore\bottle\latest


In [21]:
openvino_model_path = output_path / "weights" / "openvino" / "model.bin"
metadata = output_path / "weights" / "openvino" / "metadata.json"
print(openvino_model_path.exists(), metadata.exists())

True True


In [22]:
inferencer = OpenVINOInferencer(
    path=openvino_model_path,  # Path to the OpenVINO IR model.
    metadata=metadata,  # Path to the metadata file.
    device="CPU",  # We would like to run it on an Intel CPU.
)

In [33]:
predictions = inferencer.predict(image=image_path)
print(predictions.pred_score, predictions.pred_label)

0.1458984932711636 LabelName.NORMAL
